In [4]:
import cv2

def zoom_bbox(image_path, x_min, y_min, x_max, y_max, out_path, margin_ratio=0.25):
    img = cv2.imread(image_path)
    h, w = img.shape[:2]
    box_w = x_max - x_min
    box_h = y_max - y_min

    # Loose margin
    delta_w = int(box_w * margin_ratio)
    delta_h = int(box_h * margin_ratio)
    x_min = max(0, x_min - delta_w)
    y_min = max(0, y_min - delta_h)
    x_max = min(w, x_max + delta_w)
    y_max = min(h, y_max + delta_h)

    crop = img[y_min:y_max, x_min:x_max]
    cv2.imwrite(out_path, crop)
    print(f"Loosely cropped box at {image_path} → {out_path}")




In [5]:
# Example usage with your box coordinates
zoom_bbox(
    image_path=r"D:\GIT\JioHotstar-Ad-Vision\data\1\extracted_frames\Qantas\frame_000745.jpg",
    x_min=..., y_min=..., x_max=..., y_max=...,    # <-- Fill with your actual box for "Qantas"
    out_path=r"D:\GIT\JioHotstar-Ad-Vision\frame_000745_crop.jpg",
    margin_ratio=0.25
)

TypeError: unsupported operand type(s) for -: 'ellipsis' and 'ellipsis'